## Manipulation Check of the self-reported stress data

#### package import

In [ ]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
# "Extra package"
import pingouin as pg
# Vallat, R. (2018). Pingouin: statistics in Python. Journal of Open Source Software, 3(31), 1026,
# https://doi.org/10.21105/joss.01026

#### dataset import

In [ ]:
# change your_filename to the file name of the raw data json file

with open("your_filename.json") as jsonData:
    dataset = json.load(jsonData)

#### import a list of valid study ids after bad case/outlier removal

In [ ]:
# import the ids of the compliant study finished
with open("study_ids_without_bad_cases", "rb") as fp:
    valid_ids = pickle.load(fp)

### create a dataframe with the data for the manipulation check

In [ ]:
manipulation_check_data = {}

# loop over all datasets
for par in dataset:

    # only include the data of the selected participants
    if par in valid_ids:

        manipulation_check_data[par] = {}

        # MDBF

        # recode some items of the MDBF scale
        items_to_recode = ["MDBF_angespannt", "MDBF_nervoes", "MDBF_schlaefrig", "MDBF_ungluecklich",
                           "MDBF_unzufrieden", "MDBF_ermattet"]

        # get the data for the practice and actual condition

        # Condition
        for item in dataset[par]["Con_Mdbf"]["data"]:
            value = dataset[par]["Con_Mdbf"]["data"][item]
            if item in items_to_recode:
                manipulation_check_data[par]["Con_" + item] = 4 - value
            else:
                manipulation_check_data[par]["Con_" + item] = value

        # Practice
        for item in dataset[par]["Pr_Mdbf"]["data"]:
            value = dataset[par]["Pr_Mdbf"]["data"][item]
            if item in items_to_recode:
                manipulation_check_data[par]["Pr_" + item] = 4 - value
            else:
                manipulation_check_data[par]["Pr_" + item] = value

        # SAM

        for page in dataset[par]:
            if "_Sam_" in page:
                manipulation_check_data[par][page + "_Valenz"] = dataset[par][page]["data"]["samValence"]
                manipulation_check_data[par][page + "_Arousal"] = dataset[par][page]["data"]["samArousal"]

        # Count Task Answers

        count_answer = 0
        count_targets = 0
        for page in dataset[par]:
            if "_Count_" in page:
                targs = dataset[par][page]["data"]["Total_num_targets"]
                manipulation_check_data[par]["count_targets_" + page[9:]] = targs
                count_targets += targs
            if "_CountAns_" in page:
                ans = dataset[par][page]["data"]["Count_Task_Answer"]
                manipulation_check_data[par]["count_answer_" + page[12:]] = ans
                count_answer += ans

        manipulation_check_data[par]["Count_Total_Ans"] = count_answer
        manipulation_check_data[par]["Count_Total_Targets"] = count_targets

        # Add the condition to the Dataset
        manipulation_check_data[par]["condition"] = dataset[par]["ExperimentMetaData"]["condition"]


# create a pandas dataframe with the manipulation check data
man_check_df = pd.DataFrame(manipulation_check_data).T

# create the MDBF scales
man_check_df["Con_MDBF_GS"] = (man_check_df["Con_MDBF_wohl"] + man_check_df["Con_MDBF_gut"] + man_check_df[
    "Con_MDBF_ungluecklich"] + man_check_df["Con_MDBF_unzufrieden"]) / 4
man_check_df["Con_MDBF_RU"] = (man_check_df["Con_MDBF_ausgeglichen"] + man_check_df["Con_MDBF_ruhig"] + man_check_df[
    "Con_MDBF_angespannt"] + man_check_df["Con_MDBF_nervoes"]) / 4
man_check_df["Con_MDBF_WM"] = (man_check_df["Con_MDBF_frisch"] + man_check_df["Con_MDBF_wach"] + man_check_df[
    "Con_MDBF_schlaefrig"] + man_check_df["Con_MDBF_ermattet"]) / 4

man_check_df["Pr_MDBF_GS"] = (man_check_df["Pr_MDBF_wohl"] + man_check_df["Pr_MDBF_gut"] + man_check_df[
    "Pr_MDBF_ungluecklich"] + man_check_df["Pr_MDBF_unzufrieden"]) / 4
man_check_df["Pr_MDBF_RU"] = (man_check_df["Pr_MDBF_ausgeglichen"] + man_check_df["Pr_MDBF_ruhig"] + man_check_df[
    "Pr_MDBF_angespannt"] + man_check_df["Pr_MDBF_nervoes"]) / 4
man_check_df["Pr_MDBF_WM"] = (man_check_df["Pr_MDBF_frisch"] + man_check_df["Pr_MDBF_wach"] + man_check_df[
    "Pr_MDBF_schlaefrig"] + man_check_df["Pr_MDBF_ermattet"]) / 4

# list all manipulation check items
list(man_check_df.columns)

#### Get some info about the dataset

In [ ]:
# count the number of participants per condition
# 0 = high stress, 1 = low stress
print("Number of participants per condition:", "\n", man_check_df["condition"].value_counts())

### Save a list of all manipulation check variables to test

In [ ]:
mdbf = ["MDBF_GS", "MDBF_RU", "MDBF_WM", "stress", "nostalgia"]
sam = ['Sam_DragDrop_Arousal',
       'Sam_DragDrop_Valenz',
       'Sam_FollowBox_Arousal',
       'Sam_FollowBox_Valenz',
       'Sam_PointClick_Arousal',
       'Sam_PointClick_Valenz',
       'Sam_Slider_Arousal',
       'Sam_Slider_Valenz']

### Get and save the descriptive statistics of the manipulation check data

In [ ]:
# get descriptive statistics
descriptive_hs_data = man_check_df.loc[man_check_df["condition"] == 0].describe().sort_index(axis=1)
descriptive_ls_data = man_check_df.loc[man_check_df["condition"] == 1].describe().sort_index(axis=1)

# save the descriptive dataframes as an excel file
with pd.ExcelWriter("Descriptive_Data_Manipulation_Check.xlsx") as writer:
    descriptive_hs_data.to_excel(writer, sheet_name="High_Stress_Condition")
    descriptive_ls_data.to_excel(writer, sheet_name="Low_Stress_Condition")

### Get and save the descriptive statistics of the manipulation check data

In [ ]:
# get the Cronbach's Alpha Values of the MDBF scale
mdbf_scales = {
    "Con_GS": ["Con_MDBF_wohl", "Con_MDBF_gut", "Con_MDBF_ungluecklich", "Con_MDBF_unzufrieden"],
    "Con_RU": ["Con_MDBF_ausgeglichen", "Con_MDBF_ruhig", "Con_MDBF_angespannt", "Con_MDBF_nervoes"],
    "Con_WM": ["Con_MDBF_frisch", "Con_MDBF_wach", "Con_MDBF_schlaefrig", "Con_MDBF_ermattet"],
    "Pr_GS": ["Pr_MDBF_wohl", "Pr_MDBF_gut", "Pr_MDBF_ungluecklich", "Pr_MDBF_unzufrieden"],
    "Pr_RU": ["Pr_MDBF_ausgeglichen", "Pr_MDBF_ruhig", "Pr_MDBF_angespannt", "Pr_MDBF_nervoes"],
    "Pr_WM": ["Pr_MDBF_frisch", "Pr_MDBF_wach", "Pr_MDBF_schlaefrig", "Pr_MDBF_ermattet"]
}

for i in mdbf_scales:
    print("Cronbachs Alpha " + str(i), pg.cronbach_alpha(data=man_check_df.loc[:, mdbf_scales[i]]))

### Helper functions for the mixed anova manipulation check

In [ ]:
# prepare the dataset for the ANOVA

# helper to split the prepare the dataset for the anova and change the data from a wide to a long format
def create_anova_df(data, variable):
    # assign a subject number to each subject
    data["subject"] = np.arange(len(data))
    # replace the condition number with a string
    data["condition"].replace({0: "HS", 1: "LS"}, inplace=True)

    # change the format from wide to long
    df = pd.melt(data, id_vars=["subject", "condition"], value_vars=["Pr_" + variable, "Con_" + variable],
                 var_name="Pr-Con", value_name=variable)

    # change the name of the column to practice and condition for clearer reading of the results
    df["Pr-Con"].replace({"Pr_" + variable: "Pr", "Con_" + variable: "Con"}, inplace=True)

    return df

In [ ]:
# Create PointPlots to visualize the manipulation check results
# Tutorial here: https://raphaelvallat.com/pingouin.html
def plot_anova(data, variable):

    dataset = create_anova_df(data, variable)
    # data visualization
    sns.set()
    sns.pointplot(data=dataset, x="Pr-Con", y=variable, hue="condition", dodge=True, markers=['o', 's'],
                  capsize=.1, errwidth=1, palette='colorblind')

    plt.title("Pointplot with " + variable)
    # show the plot
    plt.show()
    # save the plot
    # plt.savefig('ANOVA' + variable + '.png')

In [ ]:
# get the anova results

def calc_anova(data, variable):

    # get the dataset
    dataset = create_anova_df(data, variable)
    # calculte the mixed anova
    aov = pg.mixed_anova(dv=variable, within="Pr-Con", between="condition", subject="subject", data=dataset)
    print("Repeated measures ANOVA with " + variable)
    # print the results in a readable table
    pg.print_table(aov)
    
    # calculate the post hoc tests
    posthocs = pg.pairwise_ttests(dv=variable, within='Pr-Con', between='condition',
                                  subject='subject', data=dataset)
    pg.print_table(posthocs)
    print("\n" + "\n")

    # return the anova and post hoc dataframes with an added index layer that is the variable name
    return pd.concat([aov], keys=[variable]), pd.concat([posthocs], keys=[variable])

### Do the manipulation Check step by step

#### 1. Plot the anova graphs

In [ ]:
# 1. plot the anova graphs for the mdbf
for i in mdbf:
    plot_anova(man_check_df, i)

In [ ]:
# 2. plot the anova graphs for the sam
for i in sam:
    plot_anova(man_check_df, i)

#### 2. Calculate the mixed anova results and save them

In [ ]:
# helper function to calc the result dataframes from a variables list
def get_anova_results(data, variable_list):

    # initialize dataframes
    anova_df = pd.DataFrame()
    posthoc_df = pd.DataFrame()

    # loop the variable list and add the result dataframes to the initialized dataframes
    for i in variable_list:

        anova, posthoc = calc_anova(data, i)

        anova_df = pd.concat([anova_df, anova])
        posthoc_df = pd.concat([posthoc_df, posthoc])

    return anova_df, posthoc_df

In [ ]:
# 3. get the manipulation check anova results for the MDBF
mdbf_anova, mdbf_posthocs = get_anova_results(man_check_df, mdbf)

# save the anova and post hoc results in an excel file
with pd.ExcelWriter("MDBF_Manipulation_Check_Results.xlsx") as writer:
    mdbf_anova.to_excel(writer, sheet_name="Mixed_Anova_Results")
    mdbf_posthocs.to_excel(writer, sheet_name="Posthoc_results")

print("Successfully saved.")

In [ ]:
# 4. get the anova results for the sam
sam_anova, sam_posthocs = get_anova_results(man_check_df, sam)

# save the manipulation check anova results for the SAM
with pd.ExcelWriter("SAM_Manipulation_Check_Results.xlsx") as writer:
    sam_anova.to_excel(writer, sheet_name="Mixed_Anova_Results")
    sam_posthocs.to_excel(writer, sheet_name="Posthoc_results")
    
print("Successfully saved.")